# Avoid Failure for Your Next Movie Date - with Data Science

## 1. Objective
When you pick the movie for your next movie date, you want to be sure to pick a good one. If you do your research you revert to the movie critics sites such as Rotten Tomatoes, Metacritic, IMDB and Fandango. 

Unfortunately [some of these site have been blamed recently](https://fivethirtyeight.com/features/fandango-movies-ratings/) that their scores are biased. Especially Fandango has been under suspicion to use a score aggregation mechanisms that put movies in a better light. As a platform that sells movie tickets, Fandango has an immediate interest in people desire to watch a movie.

In this analysis, we would like to take a data science based approach to see if this rumor holds water of not. So for your next movie date you know if you can trust the data or not and hopefully avoid failure.



## 2. The Approach and the Dataset
For this analysis we will use a data set originally developed by [FiveThirtyEight](https://fivethirtyeight.com/features/fandango-movies-ratings/). The dataset is stored in the fandango_score_comparison.csv file. It contains information on how major movie review services have rated movies.

In [ ]:
# Import data and show the head of the table
import pandas as pd
movies = pd.read_csv("fandango_score_comparison.csv")
movies.head()

Each row represents a single movie. Each column contains information about how the online movie review services `RottenTomatoes`, `Metacritic`, `IMDB`, and Fandango rated the movie. Each of these sites has 2 types of score -- User scores, which aggregate user reviews, and Critic score, which aggregate professional critical reviews of the movie. Each service puts their ratings on a different scale:

 - `RottenTomatoes` - 0-100, in increments of 1.
 - `Metacritic` - 0-100, in increments of 1.
 - `IMDB` - 0-10, in increments of .1.
 - 'Fandango' - 0-5, in increments of .5.
Typically, the primary score shown by the sites will be the Critic score. Here are descriptions of some of the relevant columns in the dataset:

| Field | Description |
| --- | --- |
| `FILM` | The name of the movie |
| `RottenTomatoes` | The RottenTomatoes (RT) critic score |
| `RottenTomatoes_User` | The RT user score |
| `Metacritic` | The Metacritic critic score |
| `Metacritic_User` | The Metacritic user score |
| `IMDB` | The IMDB score given to the movie |
| `Fandango_Stars` | The number of stars Fandango gave the movie |

To make it easier to compare scores across services, the columns were normalized so their scale and rounding matched the Fandango ratings. Any column with the suffix `_norm` is the corresponding column changed to a 0-5 scale. For example, `RT_norm` takes the `RottenTomatoes` column and turns it into a 0-5 scale from a 0-100 scale. Any column with the suffix `_round` is the rounded version of another column. For example, `RT_user_norm_round` rounds the `RT_user_norm column` to the nearest .5

For this analysis we'll primarily focus on the `Metacritic_norm_round` and the `Fandango_Stars` columns, which will let us see how `Fandango` and `Metacritic` differ in terms of review scores.

To do this, we will perform a few statistical analysis on the data to find out any potential bias.
Here the steps that we will perform:
1. Create a histogram with the score distribution and compare the two services
2. Calculate the relevant statistical indicators including standard deviation, median to describe the data sets
3. Draw a scatter plot to see if there is a correlation between the scores of the services
5. Identify the movies with the largest deviation in scores between the two services

These analysis should provide sufficient insights to see potential biases and allow us to draw conclusions.

So let's get going.

## 3. The Analysis

### 3.1 Histograms

In [ ]:
# Import Bokeh and draw the Histograms
from bokeh.plotting import figure, show, output_file

p = figure(plot_width=600, plot_height=400, title="Histogram for Movie Scores")

y, x = np.histogram(movies['Metacritic_norm_round'], bins=11, range=(0,5.5))
p.vbar(x=x[:-1], width=0.5, bottom=0,
       top=y, color="grey", alpha=0.8, legend="Metacritic Score")

y, x = np.histogram(movies['Fandango_Stars'], bins=11, range=(0,5.5))
p.vbar(x=x[:-1], width=0.45, bottom=0,
       top=y, color="blue", alpha=0.4, legend="Fandango Score")

p.legend.location = "top_left"
p.xaxis.axis_label = "Score"
p.yaxis.axis_label = "Frequency"
p.title.text_font_size = '14pt'
p.title.align = 'center'
show(p)

We clearly see that the `Fandango` scores distribution is more skewed towards higher scores whilst the `Metacitic` scores are more evenly distributed. The lowest `Fandango` score is 3 whilst `Metacritic` scores some movies with a 0.5. Let's see if the statistical metrics tell us the same.

### 3.2 Statistical Indicators

In [ ]:
# Calculate and print standard deviation, mean and median for the two datasets

score_sets = ['Metacritic_norm_round', 'Fandango_Stars']

print("Standard Deviation")
print(movies[score_sets].apply(np.std))

print("\nMean")
print(movies[score_sets].apply(np.mean))

print("\nMedian")
print(movies[score_sets].apply(np.median))



We can observe that also the metrics tell the same story. `Fandango` mean is above 4 and the standard deviation is smaller. This clearly shows `Fandango` scores are concentrated on the higher end compared the `Metacritic`. However this does not yet tell us if at least withing the more narrow range, `Fandango` is rating movies relative in a similar way as `Metacritic`. In other words, we need to see if the scores correlate.

### 3.3 Scatter Plot

Let's create a scatter plot to see ho well the scores correlate.

In [ ]:
# Create scatter diagram
from bokeh.models import Range1d, Label
p = figure(plot_width=500, plot_height=500, title="Fandango vs. Metacritic Scores")
x1 = movies["Metacritic_norm_round"]
y1 = movies["Fandango_Stars"]
p.circle(x1, y1, size=20, alpha=0.2)

#Find linear regression parameters
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(x1, y1)
x2 = np.arange(0,7,1)
y2 = intercept + slope*x2
p.line(x2,y2)


p.xaxis.axis_label = "Metacritic Score"
p.yaxis.axis_label = "Fandango Score"
p.x_range = Range1d(0, 5.5)
p.y_range = Range1d(0, 5.5)
r_text = "r value = " + str(r_value)
r_tag = Label(x=0.3, y=3.5, text=r_text, render_mode='css',
               background_fill_color='white', background_fill_alpha=0.3)
p.add_layout(r_tag)
p.title.text_font_size = '14pt'
p.title.align = 'center'
show(p)


Since all the scores are discrete values of 0, 0.5, 1, 1.5, ... 5, we cannot see a typical scatter chart but instead the dots line up on a grid. As a result, many of the dots overlap and the darker the dot, the more dots overlap.

We can observe that the correlation between the two movie rating services scores is not very high. We see the dots very scattered all over the chart. The Pearson r value is low and the slope of the fitted line is very low.

It seems that there is a large difference between the `Fandango` and the `Metacritic` scores.

Before we get to the final conclusion, let's look at which movies have a high difference between the `Fandango` and the `Metacritic` score.

### 3.4 Find Movies with the Largest Score Deviation

In [ ]:
# Calculate the difference between the scores, sort an print the top 10
movies["difference"] = abs(movies["Fandango_Stars"] - movies["Metacritic_norm_round"])
deviations = movies.sort_values("difference", 
                   ascending=False)[["FILM", "difference", "Fandango_Stars", "Metacritic_norm_round"]]
deviations.head(10)

We can see that the 10 movies with the largest deviation of scores are all rated higher on Fandango. There are a number of movies wit ha difference of 3 points or more between `Metacritic` and `Fandango`. This clearly indicates that there is something off with the Fandango scores.

## 4. Conclusion

The analysis clearly shows that the `Fandango` movies ratings are skewed to more positive ratings. The distribution shows that the Fandango scores are at the higher end. Beyond that, the `Fandango` score has only very little correlation with the `Metacritic` score and there are massive difference between the two scoring services. Possibly this is to aid movie ticket sales on `Fandango`.

Based on this insight, we can not advise to use `Fandango` to pick the movie for your next movie date. Chances are that your movie partner will be disappointed by your movie picking skills. So if you want to make your next movie date a success, better use one of the other movie rating services. 